**step 2:** prediction model building 

The following imports are done because all previous functions were copied to python script files for easier navigation between notebooks, in this second step we will focus on using the data we cleaned to model predictions of the 'risk_score' parameter.

In [1]:
from src.pipeline import pipeline
from src.preprocess import BeforePipeline
from src.map import data_mapping
from src.feature_engineering import add_engineered_features
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import pandas as pd
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance
from src.config import cols_to_drop

37


In [2]:
data_path = '../data/raw/data_chunck.parquet'
l = ['settlement_date', 'debt_settlement_flag_date', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'next_pymnt_d', 'out_prncp_inv ', 'total_pymnt_inv ', 'settlement_status', 'hardship_type', 'hardship_loan_status', 'hardship_status', 'hardship_reason', 'debt_settlement_flag_Y', 'debt_settlement_flag_N', 'verification_status_joint', 'sec_app_earliest_cr_line']

In [11]:
def all_data_prep(n): #a function to quickly get how much data we need 
    df = pd.read_parquet(data_path).sample(n=n, random_state=2)
    print(f'Shape before data prep: {df.shape}')

    bp = BeforePipeline()
    df = add_engineered_features(df)

    print(set(l) in set(df.columns))
    print("Columns present before drop:", list(df.columns))
    print("'debt_settlement_flag_N' in data:", 'debt_settlement_flag_N' in df.columns)
    print("'debt_settlement_flag_N ' (with space) in data:", 'debt_settlement_flag_N ' in df.columns)
    df = bp.all_before_pipeline(df)
    
    df_y = np.log1p(df['risk_score']) #we log transform y to help the model
    df = df.drop(columns='risk_score')

    print(f'shape after data prep: {df.shape}')
    return df,df_y

### Feature Engineering
We will try to make new features that help the model generalize better by using existing ones.
here are all the features we will add:

Variants of debt to in income ratio:
- revol_bal_to_income = revol_bal / annual_inc
- loan_to_income = loan_amnt / annual_inc
- bc_limit_to_income = total_bc_limit / annual_inc

Credit utilization refinements:
- total_utilization = revol_bal / total_rev_hi_lim
- bc_to_total_limit_ratio = total_bc_limit / total_rev_hi_lim

New Metrics:
- tot_cur_bal_to_income = tot_cur_bal / annual_inc

Credit history features:
- recent_account_ratio = acc_open_past_24mths / total_acc

Risk Buckets & Flags:
- high_util_flag = (revol_util > 80).astype(int)
- short_history_flag = (credit_history_length < 24).astype(int)
- many_inquiries_flag = (inq_last_6mths >= 3).astype(int)


## Introduction to model building
Let's get started with model building, as we are dealing with very high dimensionality data, a lot of non linearity and mixed-type features, the optimal choice for us will probably be a tree based model, We will use a Random Forest Regressor, which is an application of bagging to decision trees with the additional “random subspace” trick: we create a large number of trees and each tree is trained on a bootstrap sample of the data chosen randomly with replacement, and at each split only a portion of the features are randomly chosen without replacement, generally $\sqrt{n}$ features for classification or $n/3$ for regression for $n$ features. 

This will save a ton of computing power while reducing the variance of our model.

In fact, if each tree result is a random variable, let's say we have $n$ random variables ${X_i}$ equally distributed but not necessarly independant, our model output $\tilde{X}$ would then be the average of all the results of the different trees:
$\tilde{X} = \frac{1}{n}\sum_{i=1}^{n} {X_i}$ if we let $Var({X_i}) = \sigma^{2}$ for all $i$ and $Cov({X_i},{X_j}) = \rho \sigma^{2}$ for all $i \neq j$ ($\rho$ being the coeificient of correlation),

Then, by a simple variance calculation we have that, $$Var(\tilde{X}) = \rho \sigma^{2} + \frac{1 - \rho}{n}\sigma^{2}$$
As n grow larger, $\frac{1 - \rho}{n}\sigma^{2} \xrightarrow[n \to \infty]{} 0$, we are then left with $Var(\tilde{X}) = \rho \sigma^{2} \leq  \sigma^{2}$ as $\rho \leq 1$.

So, the more trees we have, the more we reduce our model's output variance, this variance reduction is crucial for improving model stability, especially when evaluating the model across different economic scenarios to assess its robustness.

## Additional Note:
- In classification, the Random Forest aggregates by majority voting (which is equivalent to averaging the 0/1 votes and then thresholding) or by averaging the class probabilities. The variance formula applies to the probability estimates, which can then be thresholded to get the class label. This variance reduction in the probability estimates leads to more stable predictions.

In [12]:
#data collection
X,y = all_data_prep(n=1000_000)

print("Infinite values after cleaning:", np.isinf(X.select_dtypes(include=[np.number])).sum().sum())
print("NaN values after cleaning:", X.isnull().sum().sum())
#baseline scikit-learns random forest
rf = RandomForestRegressor(random_state=1,n_jobs=-1)
pipe = pipeline(model=rf, ver=True)


print('total_pymnt_inv' in X.columns)
#cross validation
"""
scores = cross_val_score(estimator=pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')
"""

Shape before data prep: (1000000, 148)
False
Columns present before drop: ['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joi

d:\pythonProjects\loan_default_prediction\app\src\map.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['risk_score']= data['loan_status'].map(mapping)


===> data_mapping called
===> num_data_prep called
number of cols to drop: 37
data shape before dropping: (999986, 145)
Columns to drop present in data: 27
Columns NOT found in data (cannot drop): ['next_pymnt_d', 'settlement_status', 'hardship_type', 'hardship_loan_status', 'hardship_status', 'hardship_reason', 'debt_settlement_flag_Y', 'debt_settlement_flag_N', 'verification_status_joint', 'sec_app_earliest_cr_line']
27 columns dropped successfully
data shape after dropping: (999986, 118)
===> drop_useless called
===> clean infinites called
shape after data prep: (999986, 117)
Infinite values after cleaning: 0
NaN values after cleaning: 65354
False


"\nscores = cross_val_score(estimator=pipe, X=X, y=y, cv=3, scoring='r2')\nprint(f'cross validation scores: {scores}\nmean: {scores.mean()}')\n"

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X,y)

pipe.fit(X_train, y_train)
print('model fitted')
X_trans = pipe['features'].fit_transform(X_val)
print('transformed')
perms = PermutationImportance(pipe[1], scoring='r2', n_iter=3, random_state=1).fit(X_trans, y_val)

[ColumnTransformer] ......... (1 of 5) Processing scale, total=   2.4s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.1s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.1s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.4s
[Pipeline] .......... (step 1 of 2) Processing features, total=   4.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=27.9min
model fitted
[ColumnTransformer] ......... (1 of 5) Processing scale, total=   1.0s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.2s
transformed


In [14]:
ohe_cols = pipe[0]['OHE'].get_feature_names_out().tolist()
num_cols = [col for col in X_val.columns if X_val[col].dtype != 'object']

feature_names = num_cols + ['purpose','addr_state'] + ['emp_length'] + ['earliest_cr_line'] + ohe_cols

eli5.show_weights(perms, feature_names=feature_names)

Weight,Feature
0.1694 ± 0.0006,settlement_amount
0.0294 ± 0.0007,loan_to_income
0.0256 ± 0.0009,max_bal_bc
0.0254 ± 0.0002,tot_hi_cred_lim
0.0252 ± 0.0003,term_ 36 months
0.0229 ± 0.0007,recent_account_ratio
0.0219 ± 0.0007,installment
0.0208 ± 0.0010,term_ 60 months
0.0152 ± 0.0008,bc_open_to_buy
0.0143 ± 0.0008,total_bc_limit


In [ ]:
scores = cross_val_score(estimator=pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

[ColumnTransformer] ......... (1 of 5) Processing scale, total=   2.9s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.1s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.1s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.4s
[Pipeline] .......... (step 1 of 2) Processing features, total=   4.7s
[Pipeline] ............. (step 2 of 2) Processing model, total=25.6min
[ColumnTransformer] ......... (1 of 5) Processing scale, total=   2.5s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.1s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.1s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.1s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.5s
[Pipeline] .......... (step 1 of 2) Processing features, total=   4.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=35.3mi